# Question 2

## Question 2.1

In [1]:
# symbolic computation tools
import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify

import numpy as np

from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

import mpmath


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
    


In [2]:
# We wrap in parentheses here so we can write it on multiple lines. Similar
# with the triple quotes on the string. Usually we don't need to use these things.
(theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3) = symbols("""theta_1 
                         theta_2 
                         theta_3 
                         l_1 
                         l_2 
                         l_3 """ , real = True)

In [3]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [4]:
def draw_robot(ordered_list_of_transformations):
    """
    Draw the configuration of the three-link jumper.
    """
    ax_lw = 3
    link_lw = 1

    # homogeneous unit vectors 
    x = np.array([[1, 0, 1]]).T 
    y = np.array([[0, 1, 1]]).T
    origin = np.array([[0, 0, 1]]).T

    prev_origin = np.array([[0, 0, 1]]).T    
    current_transformation = np.eye(3)

    plt.clf()
    ax = plt.gca()  # get current axis

    # now we plot
    plt.plot([prev_origin[0][0]], [prev_origin[1][0]], '-ko',linewidth=link_lw)
    plt.plot([prev_origin[0][0], x[0][0]], [prev_origin[1][0], x[1][0]], '-ro', linewidth=ax_lw)
    plt.plot([prev_origin[0][0], y[0][0]], [prev_origin[1][0], y[1][0]], '-go', linewidth=ax_lw)


    # loop the transforms in order of T_01, T_12, ... , T_N-1N
    for k, transform in enumerate(ordered_list_of_transformations):

        # update the transformation
        current_transformation = current_transformation @ transform
        new_origin = current_transformation @ origin
        new_x = current_transformation @ x
        new_y = current_transformation @ y

        # now we plot
        plt.plot([prev_origin[0][0], new_origin[0][0]], 
                 [prev_origin[1][0], new_origin[1][0]], 
                 '-ko',linewidth=link_lw)
        
        plt.plot([new_origin[0][0], new_x[0][0]], [new_origin[1][0], new_x[1][0]], '-r', linewidth=ax_lw)
        plt.plot([new_origin[0][0], new_y[0][0]], [new_origin[1][0], new_y[1][0]], '-g', linewidth=ax_lw)


#         ax.annotate('{'+str(k+1)+'}', xy=new_origin[0:2][0], xytext=new_origin[0:2][0])

        # lastly update the current_origin to the new_origin
        prev_origin = new_origin

    # lastly set the axes to be equal and appropriate
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')    

In [5]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l_1, 0)
T_23 = T(theta_3, l_2, 0)

In [6]:
FK1 = (T_01 * Matrix([[l_1], [0], [1]]))[:-1,0]
FK1

Matrix([
[l_1*cos(theta_1)],
[l_1*sin(theta_1)]])

In [7]:
FK2 = simplify((T_01 * T_12 * Matrix([[l_2], [0], [1]]))[:-1,0])
FK2

Matrix([
[l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2)],
[l_1*sin(theta_1) + l_2*sin(theta_1 + theta_2)]])

In [8]:
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[l_3], [0], [1]]))[:-1,0])
FK3

Matrix([
[l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2) + l_3*cos(theta_1 + theta_2 + theta_3)],
[l_1*sin(theta_1) + l_2*sin(theta_1 + theta_2) + l_3*sin(theta_1 + theta_2 + theta_3)]])

## Question 2.2

In [9]:
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J1

Matrix([
[-l_1*sin(theta_1), 0, 0],
[ l_1*cos(theta_1), 0, 0]])

In [10]:
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J2

Matrix([
[-l_1*sin(theta_1) - l_2*sin(theta_1 + theta_2), -l_2*sin(theta_1 + theta_2), 0],
[ l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2),  l_2*cos(theta_1 + theta_2), 0]])

In [11]:
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))
J3

Matrix([
[-l_1*sin(theta_1) - l_2*sin(theta_1 + theta_2) - l_3*sin(theta_1 + theta_2 + theta_3), -l_2*sin(theta_1 + theta_2) - l_3*sin(theta_1 + theta_2 + theta_3), -l_3*sin(theta_1 + theta_2 + theta_3)],
[ l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2) + l_3*cos(theta_1 + theta_2 + theta_3),  l_2*cos(theta_1 + theta_2) + l_3*cos(theta_1 + theta_2 + theta_3),  l_3*cos(theta_1 + theta_2 + theta_3)]])

## Question 2.3

In [12]:
J = Matrix([J1, J2, J3])
J

Matrix([
[                                                                    -l_1*sin(theta_1),                                                                  0,                                     0],
[                                                                     l_1*cos(theta_1),                                                                  0,                                     0],
[                                       -l_1*sin(theta_1) - l_2*sin(theta_1 + theta_2),                                        -l_2*sin(theta_1 + theta_2),                                     0],
[                                        l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2),                                         l_2*cos(theta_1 + theta_2),                                     0],
[-l_1*sin(theta_1) - l_2*sin(theta_1 + theta_2) - l_3*sin(theta_1 + theta_2 + theta_3), -l_2*sin(theta_1 + theta_2) - l_3*sin(theta_1 + theta_2 + theta_3), -l_3*sin(theta_1 + theta_2 + theta_3)],
[ l_1*cos(t

## Question 2.4

In [13]:
(m_1, 
 m_2, 
 m_3) = symbols("""m_1 
                   m_2 
                   m_3 
                   """ , real = True)


mass_xy = Matrix([[m_1, 0, 0, 0, 0, 0],
                  [0, m_1, 0, 0, 0, 0],
                  [0, 0, m_2, 0, 0, 0],
                  [0, 0, 0, m_2, 0, 0],
                  [0, 0, 0, 0, m_3, 0],
                  [0, 0, 0, 0, 0, m_3]])
mass_xy

Matrix([
[m_1,   0,   0,   0,   0,   0],
[  0, m_1,   0,   0,   0,   0],
[  0,   0, m_2,   0,   0,   0],
[  0,   0,   0, m_2,   0,   0],
[  0,   0,   0,   0, m_3,   0],
[  0,   0,   0,   0,   0, m_3]])

In [14]:
mass_matrix = simplify(J.T * mass_xy * J)
mass_matrix

Matrix([
[l_1**2*m_1 + l_1**2*m_2 + l_1**2*m_3 + 2*l_1*l_2*m_2*cos(theta_2) + 2*l_1*l_2*m_3*cos(theta_2) + 2*l_1*l_3*m_3*cos(theta_2 + theta_3) + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3) + l_3**2*m_3, l_1*l_2*m_2*cos(theta_2) + l_1*l_2*m_3*cos(theta_2) + l_1*l_3*m_3*cos(theta_2 + theta_3) + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3) + l_3**2*m_3, l_3*m_3*(l_1*cos(theta_2 + theta_3) + l_2*cos(theta_3) + l_3)],
[                                             l_1*l_2*m_2*cos(theta_2) + l_1*l_2*m_3*cos(theta_2) + l_1*l_3*m_3*cos(theta_2 + theta_3) + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3) + l_3**2*m_3,                                                                                            l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_3) + l_3**2*m_3,                              l_3*m_3*(l_2*cos(theta_3) + l_3)],
[                                                                                                                                          

## Question 2.5

In [15]:
simplify(mass_matrix.subs([(theta_1,np.pi/2), (theta_2,0), (theta_3,0)]))

Matrix([
[l_1**2*m_1 + l_1**2*m_2 + l_1**2*m_3 + 2*l_1*l_2*m_2 + 2*l_1*l_2*m_3 + 2*l_1*l_3*m_3 + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3 + l_3**2*m_3, l_1*l_2*m_2 + l_1*l_2*m_3 + l_1*l_3*m_3 + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3 + l_3**2*m_3, l_3*m_3*(l_1 + l_2 + l_3)],
[                                             l_1*l_2*m_2 + l_1*l_2*m_3 + l_1*l_3*m_3 + l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3 + l_3**2*m_3,                                           l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3 + l_3**2*m_3,       l_3*m_3*(l_2 + l_3)],
[                                                                                                                  l_3*m_3*(l_1 + l_2 + l_3),                                                                            l_3*m_3*(l_2 + l_3),                l_3**2*m_3]])

## Question 2.6

In [16]:
(g) = symbols("""g""" , real = True)

In [17]:
G_1 = J1.T * Matrix([[0], [-m_1 * g]])
G_1

Matrix([
[-g*l_1*m_1*cos(theta_1)],
[                      0],
[                      0]])

In [18]:
G_2 = J2.T * Matrix([[0], [-m_2 * g]])
G_2

Matrix([
[-g*m_2*(l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2))],
[                     -g*l_2*m_2*cos(theta_1 + theta_2)],
[                                                     0]])

In [19]:
G_3 = J3.T * Matrix([[0], [-m_3 * g]])
G_3

Matrix([
[-g*m_3*(l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2) + l_3*cos(theta_1 + theta_2 + theta_3))],
[                   -g*m_3*(l_2*cos(theta_1 + theta_2) + l_3*cos(theta_1 + theta_2 + theta_3))],
[                                                  -g*l_3*m_3*cos(theta_1 + theta_2 + theta_3)]])

In [20]:
G = simplify(G_1 + G_2 + G_3)
G

Matrix([
[-g*(l_1*m_1*cos(theta_1) + l_1*m_2*cos(theta_1) + l_1*m_3*cos(theta_1) + l_2*m_2*cos(theta_1 + theta_2) + l_2*m_3*cos(theta_1 + theta_2) + l_3*m_3*cos(theta_1 + theta_2 + theta_3))],
[                                                                     -g*(l_2*m_2*cos(theta_1 + theta_2) + l_2*m_3*cos(theta_1 + theta_2) + l_3*m_3*cos(theta_1 + theta_2 + theta_3))],
[                                                                                                                                         -g*l_3*m_3*cos(theta_1 + theta_2 + theta_3)]])

## Question 2.7

In [21]:
simplify(G.subs([(theta_1,np.pi/2), (theta_2,0), (theta_3,0)]))

Matrix([
[-6.12323399573677e-17*g*(l_1*m_1 + l_1*m_2 + l_1*m_3 + l_2*m_2 + l_2*m_3 + l_3*m_3)],
[                              -6.12323399573677e-17*g*(l_2*m_2 + l_2*m_3 + l_3*m_3)],
[                                                    -6.12323399573677e-17*g*l_3*m_3]])

The calculated torques have some rounding error. The torques from gravity in the standing position are zero.